In [4]:
import pandas as pd
import numpy as np

In [5]:
zillow_df = pd.read_csv('zillow-carlifornia.csv')

<h3>Initial Exploration </h3>

In [6]:
zillow_df.head()

,Date,MedianSoldPrice_AllHomes.California,MedianMortageRate,UnemploymentRate
0,2008-02-29,470000.0,5.29,6.3
1,2008-03-31,441000.0,5.44,6.2
2,2008-04-30,460000.0,5.42,6.4
3,2008-05-31,429000.0,5.47,6.3
4,2008-06-30,437500.0,5.60,6.2


In [7]:
zillow_df.shape

(107, 4)

In [8]:
zillow_df.describe()

,MedianSoldPrice_AllHomes.California,MedianMortageRate,UnemploymentRate
count,95.000000,107.000000,107.000000
mean,399105.263158,4.186542,6.823364
std,45971.423521,0.636049,1.183979
min,335000.000000,3.290000,4.700000
25%,360125.000000,3.780000,6.100000
50%,375000.000000,4.060000,7.000000
75%,440000.000000,4.430000,7.650000
max,489000.000000,6.030000,9.100000


In [9]:
zillow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 4 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Date                                 107 non-null    object 
 1   MedianSoldPrice_AllHomes.California  95 non-null     float64
 2   MedianMortageRate                    107 non-null    float64
 3   UnemploymentRate                     107 non-null    float64
dtypes: float64(3), object(1)
memory usage: 3.5+ KB
